In [3]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import logging as lg
lg.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=lg.INFO)
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

# General column meanings
* y_org: what the orginal time series data is (should have missing values staying as missing!)
* y: what the model was trained on (the y in val section is kinda irrelevant then)
* yhat_org: original unchanged output of the models predict function
* yhat: final predictions including post processing
# Scoring
* Should all be scored using y_org vs yhat

# AIM: Get new forcasts -> v2.5
* y from v2 stays as y because it's what the model was trained on (this should be shown in alphad green?)
* Get y from v1.6 into y_org
* val line (in red) should be got from y_org
* round yhat to nearest int and cast to int

## Version 2.5
NOTE: may not be properly following full column meanings due to y_org being filled with 0s

Should set version directory name in next cell. Should describe version specifics (outliers, holidays, validation period)
* TRAINING
    * Val indexing on -60
    * Cut outliers out on upper 95% quartile `forecast.loc[forecast['yhat'] < 0,['yhat']] = 0.0`
    * Linear growth
    * Fill ALL other NaNs to 0
    * Now with try:except: for the `RuntimeError: k initialized to invalid value (-nan)` which replaces first `y` with 0.001
    
* PREDICTIONS
    * Truncating predictions at 0 
    * Rounding to nearest int

	* Need this now as the outlier cutting might remove the initial 0.001 we put in there
### Remarks
* ?

In [5]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'

In [6]:
TARGET_VERSION = 'v2.5/'
BASE_VERSION = 'v2/'
Y_VERSION = 'v1.6/'
#os.makedirs(PROPHET_PATH+TARGET_VERSION)

In [7]:
full_forecast_files = [x.split('/')[-1] for x in glob.glob(PROPHET_PATH+BASE_VERSION+'*df.f')]

In [8]:
previously_processed_files = [x.split('/')[-1] for x in glob.glob(PROPHET_PATH+TARGET_VERSION+'*df.f')]

In [9]:
forecast_files = full_forecast_files# list(set(full_forecast_files) - set(previously_processed_files))

In [10]:
forecast_files[:2]

['61780df.f', '88262df.f']

In [11]:
[x[:-4] for x in forecast_files[:2]]

['61780', '88262']

In [12]:
v2df = pd.read_feather(PROPHET_PATH+BASE_VERSION+forecast_files[0])
v16df = pd.read_feather(PROPHET_PATH+Y_VERSION+forecast_files[0])

In [83]:
v25df = pd.read_feather(PROPHET_PATH+TARGET_VERSION+'61780df.f')

In [13]:
val_results = []
for file in tqdm(forecast_files):
    v2df = pd.read_feather(PROPHET_PATH+BASE_VERSION+file)
    v16df = pd.read_feather(PROPHET_PATH+Y_VERSION+file)
    # y_org to be the orginal timeseries data
    #v2df.loc[:,'y_org'] = v16df.y
    # y is already set correcly, as the training data
    # yhat_org is already set correctly (in the creating process)
    # round and cast yhat to nearest int
    v2df.loc[:,'yhat'] = v2df.yhat.round(0).astype(int)
    #v2df.to_feather(PROPHET_PATH+TARGET_VERSION+file)
    full_smape = wiki.val.smape(v16df.y, v2df.yhat)
    val_smape = wiki.val.smape(v16df[v16df['train'] == 0].y,v2df[v2df['train'] == 0].yhat)
    val_results.append((file[:-4], full_smape, val_smape))
val_results = pd.DataFrame(val_results, columns=['page_index',TARGET_VERSION[:-1]+'_full',TARGET_VERSION[:-1]+'_val'])
val_results.to_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')

100%|██████████| 145063/145063 [27:56<00:00, 86.53it/s]


In [84]:
val_results = []
for file in tqdm(forecast_files):
    v2df = pd.read_feather(PROPHET_PATH+BASE_VERSION+file)
    v16df = pd.read_feather(PROPHET_PATH+Y_VERSION+file)
    # y_org to be the orginal timeseries data
    v2df.loc[:,'y_org'] = v16df.y
    # y is already set correcly, as the training data
    # yhat_org is already set correctly (in the creating process)
    # round and cast yhat to nearest int
    v2df.loc[:,'yhat'] = v2df.yhat.round(0).astype(int)
    v2df.to_feather(PROPHET_PATH+TARGET_VERSION+file)
    full_smape = wiki.val.smape(v2df.y_org, v2df.yhat)
    val_smape = wiki.val.smape(v2df[v2df['train'] == 0].y_org,v2df[v2df['train'] == 0].yhat)
    val_results.append((file[:-4], full_smape, val_smape))
val_results = pd.DataFrame(val_results, columns=['page_index',TARGET_VERSION[:-1]+'_full',TARGET_VERSION[:-1]+'_val'])
val_results.to_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')


100%|██████████| 142346/142346 [1:47:51<00:00, 22.00it/s] 


In [85]:
resultsdf = pd.read_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')

In [87]:
resultsdf['v2.5_val'].mean()
# previous best was v2 at 58.81101798102403

55.497131263774811